<a href="https://colab.research.google.com/github/JacoboGGLeon/abd_2020_1/blob/master/Pr%C3%A1ctica_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3
## Replicación Lógica Maestro - Esclavo

### Material

* [Setting up logical replication in PostgreSQL](https://medium.com/@techrandomthoughts/setting-up-logical-replication-in-postgresql-43887b9e1d80)

* [Getting Started with PostgreSQL Streaming Replication
](https://medium.com/@kristi.anderson/getting-started-with-postgresql-streaming-replication-ade416298685)

* [Getting error: Peer authentication failed for user “postgres”, when trying to get pgsql working with rails](https://stackoverflow.com/questions/18664074/getting-error-peer-authentication-failed-for-user-postgres-when-trying-to-ge)


## En la máquina virtual

### Crear regla de puerto de entrada

* Intervalo de puesrtos de destino: `5432`
* Protocolo: `TCP`
* Nombre: `Port_5432`

<img src="https://docs.google.com/uc?export=download&id=1851ThbDiRLCpHc-tVmDSZy5cCxPgiDlc" width= "300">

### Modificamos la contraseña

`sudo -u postgres psql`

`\password postgres`

`\q`

`sudo service postgresql restart`

### Modificar los archivos `pg_hba.conf` y `postgresql.conf`

#### Para `pg_hba.conf`

`sudo nano /etc/postgresql/11/main/pg_hba.conf`

---

`# Database administrative login by Unix domain socket`

`local   all             postgres                                md5`

---

`# IPv4 local connections:`

`host    all             all             0.0.0.0/0            md5`

---

#### Para `postgresql.conf`

`sudo nano /etc/postgresql/11/main/postgresql.conf`

---

`# Add settings for extensions here`

`listen_addresses = '*'`

---

#### Reiniciamos el servicio
`sudo service postgresql restart`

### Master

#### 1. Modificamos en `postgresql.conf` para activar la replicación lógica

`sudo nano /etc/postgresql/11/main/postgresql.conf`

---

`# WRITE-AHEAD LOG`

`wal_level = logical`

---

#### 2. Reiniciamos el servicio

`sudo service postgresql restart`



#### 3. Crear una base de datos `log_rep_db`

`sudo -u postgres psql`

`CREATE DATABASE log_rep_db;`

`\l`

`\c log_rep_db`

`CREATE TABLE family (mid int, mname text);`

`CREATE ROLE log_rep_user WITH REPLICATION LOGIN PASSWORD 'log_rep_user';`

`GRANT ALL PRIVILEGES ON DATABASE log_rep_db TO log_rep_user;`

`GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO log_rep_user;`

#### 5. Crear una Replicación Física
`CREATE PUBLICATION mypub1;`

`ALTER PUBLICATION mypub1 ADD TABLE family;`


#### 7. Insertamos datos
`sudo -u postgres psql`

`\c log_rep_db`

`INSERT INTO family (mid,mname) VALUES (1,'Vito Corleone');`

`INSERT INTO family (mid,mname) VALUES (2,'Michael Corleone');`

`INSERT INTO family (mid,mname) VALUES (3,'Walter White');`

## Slave

#### 3. Crear una base de datos log_rep_db
`sudo -u postgres psql`

`CREATE DATABASE log_rep_db;`

`\c log_rep_db`

`CREATE TABLE family (mid int,mname text);`

#### 4. Creamos una vista

`sudo -u postgres psql`

`\l`

`\c log_rep_db`

`CREATE VIEW vista AS SELECT * FROM family;`

`SELECT * FROM vista;`

#### 6. Creamos una subscripción
`\c log_rep_db`

`CREATE SUBSCRIPTION mysub1 CONNECTION 'host=52.168.89.238 port=5432 password=log_rep_user user=log_rep_user dbname=log_rep_db' PUBLICATION mypub1;`

#### 8. Hacemos una consulta

`SELECT * FROM vista;`